In [97]:
#### IMPORTS ####

# data visualisation and manipulation
import numpy as np
import pandas as pd

import random
import pickle 
##graphique
from bokeh.plotting import figure, show, output_file
from bokeh.models.tools import HoverTool
from bokeh.layouts import layout, column
from bokeh.models import ColumnDataSource, Div
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc
import ipywidgets as widgets

# fichiers
import json



In [2]:
#### Chargement des fichiers ####

# Fichier json d'apprentissage
with open('train.json', 'r') as f1:
    datas1 = json.load(f1) # id, cuisine, ingredients[]

# Fichier json dont on cherche les types de cuisines
with open('test.json', 'r') as f2:
    datas2 = json.load(f2) # id, ingredients[]


In [3]:
#### Extraction des données des fichiers ####
csv = []

for data in datas1:
    for ingredient in data['ingredients']:
        # Ajout de la ligne id-cuisine-ingredient
        tmp = dict()
        tmp['id'] = str(data['id'])
        tmp['cuisine'] = data['cuisine']
        tmp['ingredient'] = ingredient
        csv.append(tmp)

df = pd.DataFrame(csv) # DataFrame des données Source (d'apprentissage : train.json)
#dft = pd.DataFrame(csv2) # DataFrame des données de Test (test.json)

In [4]:
#### Récupération des données extraites ####
cuisines = list(df['cuisine'].unique())

In [9]:
#### Ouverture de nouveaux fichiers ####
    #fichier dans lequel on a nos coordonnés de nos recettes par pays
file = open('points_par_pays.data', 'rb')
points_par_pays = pickle.load(file)
file.close()
    #fichier dans lequel on a tout les ID des recettes 
file2 = open('id_par_points_par_pays.data', 'rb')
id_recette = pickle.load(file2)
file2.close()

In [59]:
#### genere des couleurs aléatoires ####
def couleur_generer():
    number_of_colors = 20
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]
    return color

In [77]:
#### affichage de nos recettes sur une carte ####
#retourne la source de tous nos points afin de les afficher 
def ajouter(tout_les_points):
    color = couleur_generer()
    #on crée d'abord notre source à affichera avec un dataframe 
    table = []
    tmp = dict()
    for i in range (len(tout_les_points)):
        mes_points = np.asarray(tout_les_points[i])
        for j in range(len(mes_points)):
            tmp['color'] = color[i];
            tmp['cuisine']= cuisines[i]
            tmp = dict()
            tmp['x'] = mes_points[j][0]
            tmp['y'] = mes_points[j][1]
            tmp['id'] = id_recette[i][j]
            tmp['ingredient']=""
            table.append(tmp)

    df_c = pd.DataFrame(table)
    return df_c.fillna(0)
     
    

In [78]:
#affiche les points de tous nos points
def afficherCarte(dataSource):
    #on prépare la carte à afficher 
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS,  tooltips = [ ('id', '@id'),('cuisine', '@cuisine'), ('ingrédients', '@ingredient')])
    p.title.text = 'Carte des plats par couleur de pays'
    source = ColumnDataSource(dataSource) #on installe notre source pour l'afficher 
    p.circle( 'x' , 'y', source= source, fill_color= 'color' , fill_alpha= 1, line_color=None, size=10 )
    show(p)

In [80]:
#Nécessite la source de tous nos points pour mettre en valeur un de ces points 
def mettre_en_valeur_recette(dataSource, monPoint):
    df_test = dataSource.loc[dataSource['id'] == monPoint]
    return df_test
#affichage de nos points + le point qu'on veut mettre en valeur dans un graphe
def afficherCarte2(DS_tout_les_points, DS_monPoint):
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS,  tooltips = [ ('id', '@id'),('cuisine', '@cuisine'), ('ingrédients', '@ingredient')])
    p.title.text = 'Carte des plats par couleur de pays'
    source = ColumnDataSource(DS_tout_les_points) #on installe notre source pour l'afficher 
    p.circle( 'x' , 'y', source= source, fill_color= 'color' , fill_alpha= 1, line_color=None, size=10 )
    source2 = ColumnDataSource(DS_monPoint)
    p.circle( 'x' , 'y', source= source2, fill_color= 'color' , fill_alpha= 1, line_color=None, size=30)
    show(p)

In [92]:
#affichage d'un tout nouveau point avec ses coordonnées en parametre 
def afficherNouveauPoint(DS_tout_les_points, coordonnee): #[50 ,20]
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS,  tooltips = [ ('id', '@id'),('cuisine', '@cuisine'),('ingrédients', '@ingredient')])
    p.title.text = 'Carte des plats par couleur de pays'
    source = ColumnDataSource(DS_tout_les_points) #on installe notre source pour l'afficher 
    p.circle( 'x' , 'y', source= source, fill_color= 'color' , fill_alpha= 1, line_color=None, size=10 )
    p.circle( coordonnee[0] , coordonnee[1],  fill_color= 'red' , fill_alpha= 1, line_color=None, size=30)
    show(p)
    

In [95]:
df_c = ajouter(points_par_pays)
afficherNouveauPoint(df_c, coordonnee)